In [16]:
import pandas as pd
from scipy.spatial.distance import cdist
import numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta

In [17]:
df = pd.read_csv('data/locations_data.csv')
df['date'] = df['date'].astype('datetime64[ns, Europe/Warsaw]')

In [18]:
stops = pd.read_csv('data/stops.txt')

In [19]:
stop_radius = 0.001
batch_size = 500000 # more should be faster, too much will overload ram

df.loc[:, 'at_stop'] = False
df.loc[:, 'stop'] = None
df.loc[:, 'stop_dist'] = 0. # for dubugging

stop_coords = stops[['stop_lat', 'stop_lon']].to_numpy()

for i in tqdm(range(0, len(df), batch_size)):
    batch = df.iloc[i:i + batch_size]
    coords = batch[['x', 'y']].to_numpy()
    dists = cdist(coords, stop_coords)
    at_stop = dists.min(1) < stop_radius
    batch.loc[:, 'at_stop'] = at_stop
    batch.loc[at_stop, 'stop'] = dists.argmin(1)[at_stop]
    batch.loc[at_stop, 'stop_dist'] = dists.min(1)[at_stop] # for debugging
    df.iloc[i:i + batch_size] = batch

  0%|          | 0/7 [00:00<?, ?it/s]C:\Users\proto\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
100%|██████████| 7/7 [00:51<00:00,  7.33s/it]


In [20]:
df = df[df['at_stop']]
df['stop_name'] = df['stop'].apply(lambda x: stops['stop_name'][x])

In [21]:
def find_changes(x: pd.Series):
    return x.ne(x.shift())

In [22]:
df[df['k'] == df['k'][0]]['stop'].apply(lambda x: stops.iloc[x][['stop_id','stop_name']])[:15]

,stop_id,stop_name
0,3891,Hallera
144,3891,Hallera
288,3891,Hallera
432,3891,Hallera
577,3903,Hallera
720,3903,Hallera
863,3904,Hallera
1006,3904,Hallera
1149,3904,Hallera
1293,3814,Racławicka (szkoła)


In [23]:
stop_cols = ['stop_name', 'start', 'end', 'line', 'k']
stop_times = []
trips_cols = ['source_name', 'destination_name', 'start_time', 'end_time', 'line', 'k']
trips = []

In [27]:
def datetime_mean(a, b):
    if a > b:
        t1 = b
        t2 = a
    else:
        t1 = a
        t2 = b
    return t1 + ((t2 - t1) / 2)

def datetime_mean2(row):
    return datetime_mean(row.start, row.end)

In [26]:
for k in tqdm(df['k'].unique()):
    kdf = df[df['k'] == k]
    changes = find_changes(kdf['stop_name'])
    idxs = np.flatnonzero(changes)
    new_rows = []
    for i in range(len(idxs)):
        if i == len(idxs) - 1:
            end_idx = len(kdf) - 1
        else:
            end_idx = idxs[i + 1] - 1
        start_row = kdf.iloc[idxs[i]]
        end_row = kdf.iloc[end_idx]
        new_rows.append([start_row['stop_name'], start_row['date'], end_row['date'], start_row['name'], k])
    new_rows = pd.DataFrame(new_rows, columns=stop_cols)
    stop_times.append(new_rows)
stop_times = pd.concat(stop_times, ignore_index=True)

100%|██████████| 10144/10144 [01:11<00:00, 141.74it/s]


In [29]:
stop_times['mid'] = stop_times[['start', 'end']].apply(datetime_mean2, axis=1)

In [31]:
stop_times['stop_duration'] = stop_times['end'] - stop_times['start']

In [44]:
stop_times[stop_times['k'] == stop_times['k'].unique()[1501]]

,stop_name,start,end,line,k,mid,stop_duration
33635,KRZYKI,2022-01-10 06:10:50+01:00,2022-01-10 06:11:50+01:00,17,19265172,2022-01-10 06:11:20+01:00,0 days 00:01:00
33636,Radio i Telewizja,2022-01-10 06:12:00+01:00,2022-01-10 06:12:40+01:00,17,19265172,2022-01-10 06:12:20+01:00,0 days 00:00:40
33637,Przyjaźni,2022-01-10 06:12:50+01:00,2022-01-10 06:16:40+01:00,17,19265172,2022-01-10 06:14:45+01:00,0 days 00:03:50
33638,Braterska,2022-01-10 06:16:50+01:00,2022-01-10 06:17:10+01:00,17,19265172,2022-01-10 06:17:00+01:00,0 days 00:00:20
33639,Sąsiedzka,2022-01-10 06:17:20+01:00,2022-01-10 06:17:40+01:00,17,19265172,2022-01-10 06:17:30+01:00,0 days 00:00:20
33640,Skarbowców,2022-01-10 06:17:50+01:00,2022-01-10 06:18:40+01:00,17,19265172,2022-01-10 06:18:15+01:00,0 days 00:00:50
33641,KLECINA,2022-01-10 06:18:50+01:00,2022-01-10 06:19:30+01:00,17,19265172,2022-01-10 06:19:10+01:00,0 days 00:00:40


In [48]:
stop_times.iloc[0]['stop_name']

'Hallera'

In [11]:
# for k in tqdm(df['k'].unique()):
#     k_mask = df['k'] == k
#     changes = find_changes(df[k_mask]['stop_name'])
#     idxs = np.flatnonzero(changes)
#     new_rows = []
#     for i in idxs[1:]:
#         prev_row, cur_row = df.loc[k_mask].iloc[i-1], df.loc[k_mask].iloc[i]
#         if prev_row['name'] == cur_row['name']:
#             new_rows.append([prev_row['stop_name'], cur_row['stop_name'], prev_row['date'], cur_row['date'], cur_row['name'], k])
#     new_rows = pd.DataFrame(new_rows, columns=trips_cols)
#     trips.append(new_rows)
# trips = pd.concat(trips, ignore_index=True)

100%|██████████| 10144/10144 [07:51<00:00, 21.51it/s]


In [51]:
time_threshold = timedelta(minutes=5)

for k in tqdm(stop_times['k'].unique()):
    kdf = stop_times[stop_times['k'] == k]
    new_rows = []
    for i in range(len(kdf) - 1):
        start_stop, end_stop = kdf.iloc[i], kdf.iloc[i+1]
        if start_stop['stop_duration'] > time_threshold:
            start_time = start_stop['end']
        else:
            start_time = start_stop['mid']
        if end_stop['stop_duration'] > time_threshold:
            end_time = end_stop['start']
        else:
            end_time = end_stop['mid']
        new_rows.append([start_stop['stop_name'], end_stop['stop_name'], start_time, end_time, start_stop['line'], k])
    new_rows = pd.DataFrame(new_rows, columns=trips_cols)
    trips.append(new_rows)
trips = pd.concat(trips, ignore_index=True)

100%|██████████| 10144/10144 [01:11<00:00, 142.28it/s]


In [13]:
# trips.to_csv('data/recorded_trips.csv')

In [2]:
# trips = pd.read_csv('data/recorded_trips.csv', index_col=0)

In [53]:
trips

,source_name,destination_name,start_time,end_time,line,k
0,Hallera,Racławicka (szkoła),2022-01-08 23:16:42+01:00,2022-01-08 23:17:47+01:00,126,19246880
1,Racławicka (szkoła),Modlińska,2022-01-08 23:17:47+01:00,2022-01-08 23:19:17+01:00,126,19246880
2,Modlińska,Wawrzyniaka,2022-01-08 23:19:17+01:00,2022-01-08 23:21:07+01:00,126,19246880
3,Wawrzyniaka,Chłodna,2022-01-08 23:21:07+01:00,2022-01-08 23:21:57.500000+01:00,126,19246880
4,Chłodna,Sowia,2022-01-08 23:21:57.500000+01:00,2022-01-08 23:22:57+01:00,126,19246880
...,...,...,...,...,...,...
252737,rondo Św. Ojca Pio,Ożynowa,2022-01-12 01:17:24+01:00,2022-01-12 01:18:34+01:00,259,19256282
252738,Ożynowa,Malinowa,2022-01-12 01:18:34+01:00,2022-01-12 01:19:04+01:00,259,19256282
252739,Malinowa,Wyścigowa,2022-01-12 01:19:04+01:00,2022-01-12 01:19:59.500000+01:00,259,19256282
252740,Wyścigowa,PARK POŁUDNIOWY,2022-01-12 01:19:59.500000+01:00,2022-01-12 01:20:24+01:00,259,19256282


In [54]:
routes = pd.read_csv('routes_df.csv')

In [55]:
def name_id_split(s):
    l = s.split('-')
    return ''.join(l[:-1]), l[-1]

routes['source_name'] = routes['source'].apply(lambda x: name_id_split(x)[0])
routes['source_id'] = routes['source'].apply(lambda x: name_id_split(x)[1])
routes['target_name'] = routes['target'].apply(lambda x: name_id_split(x)[0])
routes['target_id'] = routes['target'].apply(lambda x: name_id_split(x)[1])

In [56]:
source_ids = []
destination_ids = []

for row in tqdm(trips.itertuples()):
    df2 = routes[(routes['route_id'] == row.line) & (routes['source_name'] == row.source_name) & (routes['target_name'] == row.destination_name)]
    try:
        row2 = df2.iloc[0]
        source_ids.append(row2['source_id'])
        destination_ids.append(row2['target_id'])
    except IndexError:
        # print(row)
        source_ids.append(None)
        destination_ids.append(None)

trips['source_id'] = pd.Series(source_ids)
trips['destination_id'] = pd.Series(destination_ids)

252742it [04:17, 980.12it/s] 


In [57]:
trips['likely_incorrect'] = trips['source_id'].isna()

In [58]:
trips['start_time'] = trips['start_time'].apply(datetime.fromisoformat)
trips['end_time'] = trips['end_time'].apply(datetime.fromisoformat)

TypeError: fromisoformat: argument must be str

In [ ]:
# for k in tqdm(trips['k'].values):
#     ktrips = trips.loc[trips['k'] == k]

#     starts = ktrips.loc[:, 'start_time']
#     ends = ktrips.loc[:, 'end_time']

#     for i in range(len(ktrips) - 1):
#         # row1 = trips.iloc[i]
#         # row2 = trips.iloc[i+1]
#         delta = starts.iat[i+1] - ends.iat[i]
#         if delta < timedelta(minutes=2):
#             ends.iat[i] += delta / 2
#             starts.iat[i+1] -= delta / 2
#         else:
#             ends.iat[i] += timedelta(seconds=15)
#             starts.iat[i+1] -= timedelta(seconds=15)

#     starts.iat[0] -= timedelta(seconds=15)
#     ends.iat[len(ends)-1] += timedelta(seconds=15)

100%|██████████| 252742/252742 [47:03<00:00, 89.50it/s] 


In [59]:
trips['duration'] = trips['end_time'] - trips['start_time']

In [61]:
trips.iloc[10:20]

,source_name,destination_name,start_time,end_time,line,k,source_id,destination_id,likely_incorrect,duration
10,Nowowiejska,Daszyńskiego,2022-01-08 23:18:57+01:00,2022-01-08 23:20:57+01:00,11,19227664,20627,20631,False,0 days 00:02:00
11,Daszyńskiego,Mosty Warszawskie,2022-01-08 23:20:57+01:00,2022-01-08 23:21:52.500000+01:00,11,19227664,20631,20629,False,0 days 00:00:55.500000
12,Mosty Warszawskie,KROMERA,2022-01-08 23:21:52.500000+01:00,2022-01-08 23:23:57+01:00,11,19227664,20629,24002,False,0 days 00:02:04.500000
13,GRABISZYŃSKA (Cmentarz),Fiołkowa,2022-01-08 23:17:52+01:00,2022-01-08 23:20:02+01:00,11,19227689,11610,11608,False,0 days 00:02:10
14,Fiołkowa,FAT,2022-01-08 23:20:02+01:00,2022-01-08 23:21:17+01:00,11,19227689,11608,11606,False,0 days 00:01:15
15,FAT,Aleja Pracy,2022-01-08 23:21:17+01:00,2022-01-08 23:23:17.500000+01:00,11,19227689,None,None,True,0 days 00:02:00.500000
16,Aleja Pracy,Ojca Beyzyma,2022-01-08 23:23:17.500000+01:00,2022-01-08 23:24:02+01:00,11,19227689,None,None,True,0 days 00:00:44.500000
17,Ojca Beyzyma,Mielecka,2022-01-08 23:24:02+01:00,2022-01-08 23:25:08+01:00,11,19227689,None,None,True,0 days 00:01:06
18,Mielecka,Gajowicka,2022-01-08 23:25:08+01:00,2022-01-08 23:26:03+01:00,11,19227689,None,None,True,0 days 00:00:55
19,Gajowicka,Hallera,2022-01-08 23:26:03+01:00,2022-01-08 23:27:28+01:00,11,19227689,None,None,True,0 days 00:01:25


In [62]:
trips.to_csv('data/recorded_trips.csv')